# py0xcluster

py0xCluster is a package dedicated to perform exploratory data analysis and machine learning tasks on DEX activity (Decentralized Exhanges) and web3 data.

This is for now an educational project for myself with the aim of performing fun data-science projects around blockchain data gathered through the Graph Network (https://thegraph.com/)

## Target objectives

- Establish meaningful grouping of address by clustering DEX traders and LP
    - Feature Extraction:
        - TBD but based on mint/swap/burn data from messari subgraphs entities
        - with or without balances at swap time (web3py fetch balance at block)
        - EOA vs Contracts
    - Dimensionality reduction:
        - UMAP / tSNE or PCA / ICA
    - Clustering:
        - DBSCAN
        - silhouette evaluation
    - Visualization:
        - scatter plot with color-coded returns? (TBD)

## Secondary objective: identify which group has the most profitable activity

- Triggered Average of price by swap in/out by group of addresses

- Predict future returns based on the activity of previously clustered groups of addresses

## ML overall approach:

- Decide whether adopting time-series vs tabular approach (preference for the first one)
- Compute time-series based on extracted features and certain kernels / windowing
- Begin by classification approach of expected future (down-bad / neurtral / up-strong)
- Extend to regression

## Random list of potential features:

### Accounts

### Relative to a pool:
- z-scored (clarify how) difference of price 24h? after swap -> could be target independant variable

- nb of events (z-scored to other addresses on same pool)
- average swap size (z-scored/pool)
- average deposit size (z-scored/pool)
- average withdraw size (z-score/pool)

### Account only

- Total nb of positions: swapCount, depositCount, withdrawCount
- ratio? of nb of: swaps / (deposits + withdraws)

### Account - Position
- nb of (liquid) pools interacted with
- % of events (likely swaps) happening in the same block (possibly identical to MEV bots?)
- % of Limit order on uni-v3 (one deposit amout = 0)

### Account - Web3

- is contract?
- Normalized balance (compared to other users) at time of events

# Roadmap:

## Easy / To implement first

### Aggregation / Feature computation

- Aggregate unique addresses
- Implement Account-only query
- First Web3 requests (is_contract / ETH balance)

### First plots

- First features distribution
- PCA/ICA -> t-SNE

## Next, not immediate priority

- Pool clustering / identify easy-best features

### Data Management

- Store/Retrieve to/from SQLite?
- Consider parquet / feather / hdf5

## Secondary, nice to do

### Package

- Update and test requirements / setup
- Document classes and methods with nicely formatted docstrings to future build of the doc

### Performance

- Evaluate performance, profiling, and try improving inefficient / slow bits

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from py0xcluster.utils.query_utils import *
from py0xcluster.main_classes.pools import *
from py0xcluster.main_classes.pool_events import *

### Gathering data about most-active pools
    - need to adapt to take into account refactoring of queries-related methods for multiple entities at once

In [2]:
uni3pools_selector = PoolSelector(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    min_daily_volume_USD = 500000,
    min_TVL = 1000000, # Not implemented. consider removing
    start_date = (2022,10,11), 
    end_date = (2023,1,12),
    days_batch_size = 20)

uni3_pools = uni3pools_selector.create_pool_selection(stables='exclude', verbose=True)
uni3_pools.pools_df.head(50)

Queriying from 2022-10-11 00:00:00 to 2022-10-31 00:00:00
Queriying from 2022-10-31 00:00:00 to 2022-11-20 00:00:00
Queriying from 2022-11-20 00:00:00 to 2022-12-10 00:00:00
Queriying from 2022-12-10 00:00:00 to 2022-12-30 00:00:00
Queriying from 2022-12-30 00:00:00 to 2023-01-12 00:00:00
8842 lquidity pools snapshots retrieved
1688 stable pools snapshots (over 8842) have been removed
1997 illiquid pools snapshots (over 7154) have been removed 
69 pools were selected


,pool.name,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD,pool.protocol.name,pool.protocol.network,pool.id,token0.symbol,token1.symbol
0,Uniswap V3 USD Coin/Wrapped Ether 0.05%,2.818869e+08,1.838159e+08,1.000000,1405.300689,Uniswap V3,MAINNET,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH
1,Uniswap V3 Wrapped BTC/Wrapped Ether 0.05%,3.924009e+07,1.188640e+08,18712.826047,1405.300689,Uniswap V3,MAINNET,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,WETH
2,Uniswap V3 Wrapped Ether/Tether USD 0.05%,3.318446e+07,1.766933e+07,1405.300689,1.000000,Uniswap V3,MAINNET,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,USDT
3,Uniswap V3 USD Coin/Wrapped Ether 0.3%,2.092747e+07,1.377603e+08,1.000000,1405.300689,Uniswap V3,MAINNET,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,USDC,WETH
4,Uniswap V3 Wrapped BTC/Wrapped Ether 0.3%,1.093068e+07,1.465164e+08,18712.826047,1405.300689,Uniswap V3,MAINNET,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,WBTC,WETH
5,Uniswap V3 USD Coin/Wrapped Ether 0.01%,1.083670e+07,3.860127e+06,1.000000,1405.300689,Uniswap V3,MAINNET,0xe0554a476a092703abdb3ef35c80e0d76d32939f,USDC,WETH
6,Uniswap V3 Dai Stablecoin/Wrapped Ether 0.05%,7.424762e+06,6.461039e+06,1.000000,1405.300689,Uniswap V3,MAINNET,0x60594a405d53811d3bc4766596efd80fd545a270,DAI,WETH
7,Uniswap V3 Wrapped BTC/USD Coin 0.05%,6.096168e+06,3.632752e+06,18712.826047,1.000000,Uniswap V3,MAINNET,0x9a772018fbd77fcd2d25657e5c547baff3fd7d16,WBTC,USDC
8,Uniswap V3 Wrapped Ether/Tether USD 0.3%,5.550894e+06,5.445871e+07,1405.300689,1.000000,Uniswap V3,MAINNET,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH,USDT
9,Uniswap V3 ChainLink Token/Wrapped Ether 0.3%,5.086765e+06,2.020908e+07,6.317304,1405.300689,Uniswap V3,MAINNET,0xa6cc3c2531fdaa6ae1a3ca84c2855806728693e8,LINK,WETH


## Extracting all events from these pools

### Steps

- Data query:
    - Perform query for each pool, batch by days to accomodate response limit
    - Alternatively the query can be done on multiple pools. Ideally, the size of the batch should be proportional to the volume, but quite arbitrary to implement.
    - Consider whether do swaps / mints / burns separately or jointly
    - Loop and aggregate over days / pools (or batch of pools)


In [3]:
uni3_events_getter = PoolEventGetter(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    pool_ids = uni3_pools.pools_df['pool.id'][1:],
    start_date = (2022,10,1), 
    end_date = (2023,1,12),
    days_batch_size = 1
    )

# Get swaps, deposits, and withdraw from a pool.
uni3_events = uni3_events_getter.get_events(verbose=True)


pool: 0x4585fe77225b41b697c938b018e2ac67ac5a20c0
Queriying from 2022-10-01 00:00:00 to 2022-10-02 00:00:00
Queriying from 2022-10-02 00:00:00 to 2022-10-03 00:00:00
Queriying from 2022-10-03 00:00:00 to 2022-10-04 00:00:00
Queriying from 2022-10-04 00:00:00 to 2022-10-05 00:00:00
Queriying from 2022-10-05 00:00:00 to 2022-10-06 00:00:00
Queriying from 2022-10-06 00:00:00 to 2022-10-07 00:00:00
Queriying from 2022-10-07 00:00:00 to 2022-10-08 00:00:00
Queriying from 2022-10-08 00:00:00 to 2022-10-09 00:00:00
Queriying from 2022-10-09 00:00:00 to 2022-10-10 00:00:00
Queriying from 2022-10-10 00:00:00 to 2022-10-11 00:00:00
Queriying from 2022-10-11 00:00:00 to 2022-10-12 00:00:00
Queriying from 2022-10-12 00:00:00 to 2022-10-13 00:00:00
Queriying from 2022-10-13 00:00:00 to 2022-10-14 00:00:00
Queriying from 2022-10-14 00:00:00 to 2022-10-15 00:00:00
Queriying from 2022-10-15 00:00:00 to 2022-10-16 00:00:00
Queriying from 2022-10-16 00:00:00 to 2022-10-17 00:00:00
Queriying from 2022-10-

In [5]:
uni3_events['swaps']

,amountInUSD,amountOutUSD,amountIn,amountOut,from,to,timestamp,blockNumber,pool.id
0,81.066710,81.175038,6.100000e+16,4.198750e+05,0x446924c2c25ca1bcd5b5edb49abad9353f82ee61,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2022-10-01 10:18:59,15652668,0x4585fe77225b41b697c938b018e2ac67ac5a20c0
1,587.724854,588.927087,4.427279e+17,3.045568e+06,0xbcbe1f7288040f1ba05fa667af5e9b54842b6d19,0x1622f1bef90233a6993f3bc85759adf5b9d9d7b9,2022-10-01 07:55:11,15651951,0x4585fe77225b41b697c938b018e2ac67ac5a20c0
2,193.465144,193.775947,1.000000e+06,1.464568e+17,0xdce8e2b29a416280e8cd7e8baa1a1d244c50ee99,0x3d6798613616e389d7abd676a25175c825178c26,2022-09-30 23:23:35,15649412,0x4585fe77225b41b697c938b018e2ac67ac5a20c0
3,10953.063067,10924.734655,5.653395e+07,8.229567e+18,0xeca2e2d894d19778939bd4dfc34d2a3c45e96456,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,2022-10-01 05:31:35,15651237,0x4585fe77225b41b697c938b018e2ac67ac5a20c0
4,53.138588,53.181566,4.000000e+16,2.737910e+05,0x4eb8aef23173d1413c0577acbbd2c356a915950f,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2022-10-01 03:37:47,15650673,0x4585fe77225b41b697c938b018e2ac67ac5a20c0
...,...,...,...,...,...,...,...,...,...
2493961,1722.402960,1718.184547,9.962293e+06,1.291908e+18,0x5c5b24b4396e27fdfc2def6e4e6b9b063e65418d,0x1111111254eeb25477b68fb85ed929f73a960582,2023-01-10 10:31:23,16375825,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f
2493962,237.158603,236.483616,1.371678e+06,1.778093e+17,0x55dcad916750c19c4ec69d65ff0317767b36ce90,0x53222470cdcfb8081c0e3a50fd106f0d69e63f20,2023-01-10 10:46:47,16375902,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f
2493963,1730.728858,1725.858378,1.000000e+07,1.295790e+18,0x26a16cb247cd447c4e3a688c3da0c53167d95947,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2023-01-10 14:17:59,16376953,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f
2493964,8.943946,8.917942,5.173000e+04,6.705297e+15,0x05585e4315203dbacf9622b3659e2dc15a97d245,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2023-01-10 11:03:11,16375984,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f


In [77]:
uni3_events['deposits']

,amountUSD,from,to,timestamp,blockNumber,0,1,pool.id
0,5.702699447989583112621271946559613,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597,1672546583,16309467,3999999999999999546796336914,4775822800551261,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
1,5.702699447989583112621271946559613,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597,1672546583,16309467,3999999999999999546796336914,4775822800551261,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
2,914851.765461960850048108842731207,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1672528319,16307950,2218065428,549646359583,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
3,7631.111087067667674748898327938856,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1672807583,16331122,24040490,3579985456,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
4,1479.361575289250415460937779284362,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1672678103,16320383,5095577,627883205,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
...,...,...,...,...,...,...,...,...
63,47.34801450889457948824739015257494,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673285687,16370771,133568,24169363,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
64,60048.59240125194199882926006636583,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673384231,16378942,171548339,30157522440,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
65,30777.51020403673075550783963036063,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673257451,16368425,24722267,26499999181,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
66,19448295.85831377786226568728901846,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673279639,16370267,279198452,19399767114407,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35


In [6]:
uni3_events['withdraws']

,amountUSD,from,to,timestamp,blockNumber,InputTokenAmount0,InputTokenAmount1,pool.id
0,2103.420251605830514583427791156821,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0xeb17ca08e9c561d854ba91232440ae6fb148740a,1664636615,15654085,5733614,752541556338956786,0x4585fe77225b41b697c938b018e2ac67ac5a20c0
1,380.5984248697112596406431569822547,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0xc36442b4a4522e871399cd717abdd847ab11fe88,1664589647,15650193,971618,144412527968608193,0x4585fe77225b41b697c938b018e2ac67ac5a20c0
2,3303905.056620019591025814210925225,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0x56178a0d5f301baf6cf3e1cd53d9863437345bf9,1664641571,15654487,6526865704,1555830882715238815737,0x4585fe77225b41b697c938b018e2ac67ac5a20c0
3,68343.58992269882948330523469298852,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0xc36442b4a4522e871399cd717abdd847ab11fe88,1664641655,15654494,2483561,51544012077725390649,0x4585fe77225b41b697c938b018e2ac67ac5a20c0
4,1736072.188298861060954665496378991,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0x56178a0d5f301baf6cf3e1cd53d9863437345bf9,1664592455,15650425,5687424998,475236882939247333767,0x4585fe77225b41b697c938b018e2ac67ac5a20c0
...,...,...,...,...,...,...,...,...
88365,0,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f,0xa236c3d4acaf7a3f465dbc5e0acd548df3004148,1673423459,16382189,0,0,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f
88366,0,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f,0xa236c3d4acaf7a3f465dbc5e0acd548df3004148,1673423459,16382189,0,0,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f
88367,0,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f,0xa236c3d4acaf7a3f465dbc5e0acd548df3004148,1673423459,16382189,0,0,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f
88368,0,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f,0xa236c3d4acaf7a3f465dbc5e0acd548df3004148,1673423459,16382189,0,0,0x6ab3bba2f41e7eaa262fa5a1a9b3932fa161526f


In [15]:
uni3_events['deposits'].to_pickle('/home/fujiju/Documents/GitHub/py0xcluster/data/20230113_050500_deposits.pkl')

In [17]:
uni3_events['swaps'].nunique()

amountInUSD     1248508
amountOutUSD    1351632
amountIn        1101277
amountOut       1346714
from             254768
to               127814
timestamp        521411
blockNumber      521411
pool.id              68
dtype: int64

In [ ]:
data_lengths = [0, 0, 0]
empty_data = [data_length == 0 for data_length in data_lengths]
all(empty_data)

True

In [ ]:
dico = {'ac': 0 , 'asfd': 2}
len(dico)

2